<a href="https://colab.research.google.com/github/franciscogarate/utils_ucm/blob/master/notebooks/ClimateRisk_4_GIS_Tierra_de_Sabor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Geoposicionar usando geopy, mapa de calor con Folium y elaborar fichero de exposición para ClimadaApp (riesgos de inundación y tormenta)

In [2]:
!git clone https://github.com/franciscogarate/utils_ucm

fatal: destination path 'utils_ucm' already exists and is not an empty directory.


El portal de datos de información pública (https://datos.gob.es/es) dispone más de 80.000 bases de datos. Algunas de ellas pueden ser útiles a efectos de modelización de riesgos. En este caso, nos descargamos el fichero CSV de empresas de Castilla-León acogidas a la marca "Tierra de Sabor".
https://datos.gob.es/es/catalogo/a07002862-empresas-acogidas-a-la-marca-tierra-de-sabor1

In [15]:
import pandas as pd
import numpy as np
from geopy.geocoders import GoogleV3

In [16]:
from google.colab import userdata
api_key_googlev3 = userdata.get('api_key_google_v3')

In [17]:
# Pruebo que funcione
geolocator = GoogleV3(api_key=api_key_googlev3)
location = geolocator.geocode("175 5th Avenue NYC")
print((location.latitude, location.longitude))

(40.7413871, -73.9893731)


In [18]:
# Creo mi propia función:
def direcc(x):
    try:
        location = geolocator.geocode(x)
        return (location.latitude, location.longitude)
    except:
        return (0, 0)

In [19]:
df = pd.read_csv('./utils_ucm/data/empresas-acogidas-a-la-marca-tierra-de-sabor.csv', sep=';', usecols=(1,3,4,5,6))
df['Direccion_completa'] = df.apply(lambda x: str(x['Dirección'] + ', ' + x['Localidad']) + ', ' + str(x['C.P.']) + ' ' + x['Provincia'] + ' Spain', axis=1)
df['coord'] = df['Direccion_completa'].apply(lambda x: direcc(x))
df.head()

,Nombre Comercial,Dirección,C.P.,Localidad,Provincia,Direccion_completa,coord
0,BODEGA CUATRO RAYAS,CARRETERA RODILANA S/N,47491,SECA (LA),VALLADOLID,"CARRETERA RODILANA S/N, SECA (LA), 47491 VALLA...","(41.3429307, -4.896358)"
1,BODEGAS MONTE LA REINA,"CARRETERA TORO-ZAMORA KM. 436,7",49881,TORO,ZAMORA,"CARRETERA TORO-ZAMORA KM. 436,7, TORO, 49881 Z...","(41.52983380000001, -5.4987565)"
2,COMERCIAL AGRICOLA RIOJANA,AVENIDA PORTUGAL 27,47100,TORDESILLAS,VALLADOLID,"AVENIDA PORTUGAL 27, TORDESILLAS, 47100 VALLAD...","(41.5005432, -5.0035682)"
3,GRUPO COPESE,POL. INDUSTRIAL LA SALINA C/ EDUARDO CAPA 1,40480,COCA,SEGOVIA,"POL. INDUSTRIAL LA SALINA C/ EDUARDO CAPA 1, C...","(41.2131784, -4.5229204)"
4,DESPIECES Y MANIPULADOS MAVIMAR,AVENIDA NUESTRA SEÑORA DEL PRADO S/N,34429,SAN CEBRIAN DE CAMPOS,PALENCIA,"AVENIDA NUESTRA SEÑORA DEL PRADO S/N, SAN CEBR...","(42.1990242, -4.5314053)"


In [20]:
#Guardo las coordenadas en un array:
coord_riesgos = np.asarray(df.coord)
coord_riesgos

array([(41.3429307, -4.896358), (41.52983380000001, -5.4987565),
       (41.5005432, -5.0035682), (41.2131784, -4.5229204),
       (42.1990242, -4.5314053), (41.5281702, -5.368706299999999),
       (40.3471658, -5.554508999999999), (42.5428114, -5.5754832),
       (41.2899124, -4.3468794), (42.5706366, -5.5763369),
       (41.2588768, -4.2977962), (41.77809999999999, -2.486107),
       (42.4022869, -5.5408247), (41.9441612, -4.856601),
       (41.6236273, -4.0693857), (41.766436, -2.4790872),
       (41.7000442, -3.9407404), (40.5376765, -5.6776645),
       (42.5361668, -5.279722), (42.3745848, -3.733116),
       (41.5168067, -4.8671149), (41.5475933, -3.6530288),
       (41.3180091, -4.8358614), (41.4087836, -4.9742047),
       (40.9407128, -5.748277799999999), (41.5158749, -5.720976200000001),
       (41.5817202, -3.796222), (41.6415475, -4.1558553),
       (41.7624882, -3.7486234), (41.1230335, -4.2618797),
       (41.0183334, -5.5971791), (40.3579485, -5.525612799999999),
       (4

## Gráfico un mapa de calor con Folium

In [21]:
from folium import Map, plugins
from folium.plugins import HeatMap

Centro el mapa en Madrid:

In [22]:
map_cl = Map(location=[41.652, -4.729], zoom_start = 7.5)
map_cl

In [23]:
coord_riesgos = np.asarray(df.coord)

HeatMap(coord_riesgos, radius=20).add_to(map_cl)
map_cl

In [24]:
df['latitude'] = df['coord'].apply(lambda x: x[0])
df['longitude'] = df['coord'].apply(lambda x: x[1])
df.head()


,Nombre Comercial,Dirección,C.P.,Localidad,Provincia,Direccion_completa,coord,latitude,longitude
0,BODEGA CUATRO RAYAS,CARRETERA RODILANA S/N,47491,SECA (LA),VALLADOLID,"CARRETERA RODILANA S/N, SECA (LA), 47491 VALLA...","(41.3429307, -4.896358)",41.342931,-4.896358
1,BODEGAS MONTE LA REINA,"CARRETERA TORO-ZAMORA KM. 436,7",49881,TORO,ZAMORA,"CARRETERA TORO-ZAMORA KM. 436,7, TORO, 49881 Z...","(41.52983380000001, -5.4987565)",41.529834,-5.498756
2,COMERCIAL AGRICOLA RIOJANA,AVENIDA PORTUGAL 27,47100,TORDESILLAS,VALLADOLID,"AVENIDA PORTUGAL 27, TORDESILLAS, 47100 VALLAD...","(41.5005432, -5.0035682)",41.500543,-5.003568
3,GRUPO COPESE,POL. INDUSTRIAL LA SALINA C/ EDUARDO CAPA 1,40480,COCA,SEGOVIA,"POL. INDUSTRIAL LA SALINA C/ EDUARDO CAPA 1, C...","(41.2131784, -4.5229204)",41.213178,-4.522920
4,DESPIECES Y MANIPULADOS MAVIMAR,AVENIDA NUESTRA SEÑORA DEL PRADO S/N,34429,SAN CEBRIAN DE CAMPOS,PALENCIA,"AVENIDA NUESTRA SEÑORA DEL PRADO S/N, SAN CEBR...","(42.1990242, -4.5314053)",42.199024,-4.531405


In [25]:
df_exposure = pd.DataFrame([], columns=['latitude','longitude','value'])
df_exposure['latitude'] = df['latitude']
df_exposure['longitude'] = df['longitude']
# Desconozco posibles sumas aseguradas, así que indico 100.000 para todos
df_exposure['value'] = 100000
df_exposure.head(5)

,latitude,longitude,value
0,41.342931,-4.896358,100000
1,41.529834,-5.498756,100000
2,41.500543,-5.003568,100000
3,41.213178,-4.522920,100000
4,42.199024,-4.531405,100000


Guardo el fichero con el formato accesible por Climada-App (EIOPA)

In [28]:
df_exposure.to_excel('exposure_example_lat_long.xlsx', index=False)

![ClimadaApp](utils_ucm/notebooks/climada_app_data3.png)